# Find Clusters

In [3]:
#coloca o path raiz do projeto
working_path = "/home/thierry/Documents/thierry/"

In [4]:
import pandas as pd
import os
os.chdir(working_path)

from src.data_processment.stop_region_thierry import MovingCentroidStopRegionFinder

In [5]:
data = pd.read_csv("/home/thierry/Documents/thierry/data/0b2b9de31dfcb837.csv")

In [6]:
import datetime

for i in range(len(data)):
    data['date'][i] = datetime.datetime.strptime(data['date'][i], '%a %b %d %H:%M:%S GMT-03:00 %Y').day


/home/thierry/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [7]:
data = data.query('date < 3 or date > 28')
data30s = data.iloc[ 0::2 , : ]
data60s = data.iloc[ 0::4 , : ]
data120s = data.iloc[ 0::8 , : ]

In [8]:
data

,latitude,longitude,altitude,accuracy,provider,speed,timestamp,date,temperature,humidity,light,proximity,pressure,screen,audio,voluntary_id
281,-7.255000,-35.913102,0.0,68.400,fused,0.0,1567054467911,29,0.0,0.0,53.0,5.000305,0.0,True,Normal,12
282,-7.255070,-35.913067,0.0,24.906,fused,0.0,1567054484747,29,0.0,0.0,56.0,5.000305,0.0,True,Normal,12
283,-7.255070,-35.913067,0.0,47.344,fused,0.0,1567054499707,29,0.0,0.0,56.0,5.000305,0.0,True,Normal,12
284,-7.254980,-35.913096,0.0,52.400,fused,0.0,1567054513034,29,0.0,0.0,53.0,5.000305,0.0,True,Normal,12
285,-7.255205,-35.912986,0.0,56.100,fused,0.0,1567054528230,29,0.0,0.0,39.0,5.000305,0.0,True,Normal,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8310,-7.254410,-35.907887,0.0,2700.000,fused,0.0,1567464496645,2,0.0,0.0,8.0,5.000305,0.0,True,Normal,12
8311,-7.254410,-35.907887,0.0,2700.000,fused,0.0,1567464511667,2,0.0,0.0,5.0,5.000305,0.0,True,Normal,12
8312,-7.254410,-35.907887,0.0,2700.000,fused,0.0,1567464526704,2,0.0,0.0,5.0,5.000305,0.0,True,Normal,12
8313,-7.254410,-35.907887,0.0,2700.000,fused,0.0,1567464541756,2,0.0,0.0,5.0,5.000305,0.0,True,Normal,12


In [9]:
def timestamp_from_mili_to_seconds(milisseconds):
    return milisseconds / 1000

In [10]:
#r é o raio máximo que um ponto (lat, lon) pode assumir para compor uma Stop Region, é em metros
#delta_t é o intervalo de tempo  mínimo que pracisa para formar uma Stop Region

def find_clusters(user_data, r, delta_t):
    r = 50
    delta_t = 300

    if len(user_data) == 0:
        print("User data is empty")
        return

    print("FINDING STOP REGIONS")
    stop_region_finder = MovingCentroidStopRegionFinder(region_radius=r, delta_time=delta_t)

    clusters = stop_region_finder.find_clusters(user_data, verbose=False)
    print(len(clusters), "found")

    return clusters

In [11]:
data["local_time"] = timestamp_from_mili_to_seconds(data["timestamp"])
data30s["local_time"] = timestamp_from_mili_to_seconds(data["timestamp"])
data60s["local_time"] = timestamp_from_mili_to_seconds(data["timestamp"])
data120s["local_time"] = timestamp_from_mili_to_seconds(data["timestamp"])

/home/thierry/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/thierry/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/thierry/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

In [12]:
clusters = find_clusters(data, r=50, delta_t=5*60)
clusters30s = find_clusters(data30s, r=50, delta_t=5*60)
clusters60s = find_clusters(data60s, r=50, delta_t=5*60)
clusters120s = find_clusters(data120s, r=50, delta_t=5*60)

FINDING STOP REGIONS
191 found
FINDING STOP REGIONS
156 found
FINDING STOP REGIONS
130 found
FINDING STOP REGIONS
99 found


In [13]:
print("Porcentagem stop regions 30s",(len(clusters30s)/len(clusters)))
print("Porcentagem stop regions 60s",(len(clusters60s)/len(clusters)))
print("Porcentagem stop regions 120s",(len(clusters120s)/len(clusters)))

Porcentagem stop regions 30s 0.8167539267015707
Porcentagem stop regions 60s 0.680628272251309
Porcentagem stop regions 120s 0.518324607329843
